Business Understanding Questions: 
1. Which app categories have the highest average ratings and total installs?
2. How do app sizes relate to ratings or installs?
3. Which content rating is most common among top rated apps?

In [9]:
import os
os.environ['JAVA_HOME'] = '/opt/homebrew/Cellar/openjdk@17/17.0.16/libexec/openjdk.jdk/Contents/Home'
print("JAVA_HOME set to:", os.environ['JAVA_HOME'])

JAVA_HOME set to: /opt/homebrew/Cellar/openjdk@17/17.0.16/libexec/openjdk.jdk/Contents/Home


In [50]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
      .appName("GooglePlayStoreAnalysis")
      .getOrCreate()
)

In [58]:
data_path = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_fixed.csv"
df = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv(data_path)
)

In [59]:
path = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_fixed.csv"
df = spark.read.option("header",True).option("inferSchema",True).csv(path)

In [60]:
df.printSchema()
df.show(5, truncate=50)

root
 |-- googleplaystore_fixed: string (nullable = true)

+--------------------------------------------------+
|                             googleplaystore_fixed|
+--------------------------------------------------+
|                                               App|
|    Photo Editor & Candy Camera & Grid & ScrapBook|
|                               Coloring book moana|
|U Launcher Lite – FREE Live Cool Themes, Hide Apps|
|                             Sketch - Draw & Paint|
+--------------------------------------------------+
only showing top 5 rows


In [63]:
from pyspark.sql.functions import regexp_replace, col, to_date, initcap, lower, avg, round

# 4. Clean Price & Installs
df = (df
      .withColumn("Price",
         regexp_replace(col("Price"), r"[\$]", "").cast("double"))
      .withColumn("Installs",
         regexp_replace(col("Installs"), r"[+,]", "").cast("long"))
     )
df.show(5)

{"ts": "2025-07-24 16:32:01.811", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Price` cannot be resolved. Did you mean one of the following? [`googleplaystore_fixed`]. SQLSTATE: 42703", "context": {"file": "line 6 in cell [63]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o708.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Price` cannot be resolved. Did you mean one of the following? [`googleplaystore_fixed`]. SQLSTATE: 42703;\n'Project [googleplaystore_fixed#223, cast('regexp_replace('Price, [\\$], ) as double) AS Price#230]\n+- Relation [googleplaystore_fixed#223] csv\n\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.unresolvedAttributeError(QueryCompil

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Price` cannot be resolved. Did you mean one of the following? [`googleplaystore_fixed`]. SQLSTATE: 42703;
'Project [googleplaystore_fixed#223, cast('regexp_replace('Price, [\$], ) as double) AS Price#230]
+- Relation [googleplaystore_fixed#223] csv


In [66]:
import os

gps_folder = os.path.expanduser("~/Desktop/Google Play Store")
print("Folder path:", gps_folder)
print("Contents:")
for fn in sorted(os.listdir(gps_folder)):
    print(" ", fn)

Folder path: /Users/simranghai/Desktop/Google Play Store
Contents:
  .DS_Store
  .ipynb_checkpoints
  googleplaystore.ipynb
  googleplaystore_fixed.csv
  googleplaystore_fixed.numbers


In [67]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GPS").getOrCreate()

data_path = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_fixed.csv"
df = spark.read.option("header", True) \
               .option("inferSchema", True) \
               .csv(data_path)

25/07/24 16:34:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [68]:
df.printSchema()
df.show(5, truncate=50)

root
 |-- googleplaystore_fixed: string (nullable = true)

+--------------------------------------------------+
|                             googleplaystore_fixed|
+--------------------------------------------------+
|                                               App|
|    Photo Editor & Candy Camera & Grid & ScrapBook|
|                               Coloring book moana|
|U Launcher Lite – FREE Live Cool Themes, Hide Apps|
|                             Sketch - Draw & Paint|
+--------------------------------------------------+
only showing top 5 rows


In [69]:
from pyspark.sql.functions import col, count, when

print("Total rows:", df.count())
df.select([ count(when(col(c).isNull(), c)).alias(c) 
            for c in df.columns ]) \
  .show(truncate=False)

Total rows: 10842
+---------------------+
|googleplaystore_fixed|
+---------------------+
|0                    |
+---------------------+



In [70]:
from pyspark.sql.functions import regexp_replace

df = (df
      .withColumn("Price",    regexp_replace(col("Price"),    r"[\$]", "").cast("double"))
      .withColumn("Installs", regexp_replace(col("Installs"), r"[+,]", "").cast("long"))
     )
df.select("Price","Installs").show(5)

{"ts": "2025-07-24 16:35:10.398", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Price` cannot be resolved. Did you mean one of the following? [`googleplaystore_fixed`]. SQLSTATE: 42703", "context": {"file": "line 4 in cell [70]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o801.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Price` cannot be resolved. Did you mean one of the following? [`googleplaystore_fixed`]. SQLSTATE: 42703;\n'Project [googleplaystore_fixed#248, cast('regexp_replace('Price, [\\$], ) as double) AS Price#271]\n+- Relation [googleplaystore_fixed#248] csv\n\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.unresolvedAttributeError(QueryCompil

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Price` cannot be resolved. Did you mean one of the following? [`googleplaystore_fixed`]. SQLSTATE: 42703;
'Project [googleplaystore_fixed#248, cast('regexp_replace('Price, [\$], ) as double) AS Price#271]
+- Relation [googleplaystore_fixed#248] csv


In [73]:
import os, glob

gps_dir = "/Users/simranghai/Desktop/Google Play Store"

# Find all CSV files in the folder
csv_files = glob.glob(os.path.join(gps_dir, "*.csv"))
print("CSV files in folder:")
for f in csv_files:
    print(" ", os.path.basename(f))

# For each CSV, print its first two lines so we can see the header
print("\nPreview headers:")
for f in csv_files:
    print("\n>>>", os.path.basename(f))
    with open(f, "r", encoding="utf-8") as fh:
        for _ in range(2):
            print("  ", fh.readline().strip())

CSV files in folder:
  googleplaystore_fixed.csv

Preview headers:

>>> googleplaystore_fixed.csv
   googleplaystore_fixed
   App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver


# Google Play Store Analysis

## Analytical Questions
1. **Which app categories have the highest average ratings and total installs?**
2. **Is there a relationship between app price (free vs. paid) and rating or installs?**
3. **Which content rating (e.g., Everyone, Teen, Mature) is most common among top-rated apps?**
4. **How do app sizes relate to ratings or installs?**
5. **Which categories generate the most paid apps, and do these paid apps have better ratings than free apps?**

In [74]:
import pandas as pd, os

# Paths
raw   = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_fixed.csv"
clean = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_clean.csv"

# Read everything **except** the first row, then save
pd.read_csv(raw, skiprows=1).to_csv(clean, index=False)

print("Wrote clean file:", clean)
print("Exists? ", os.path.exists(clean))

Wrote clean file: /Users/simranghai/Desktop/Google Play Store/googleplaystore_clean.csv
Exists?  True


In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_date, initcap, lower, avg, round

spark = SparkSession.builder.appName("GPS").getOrCreate()

path = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_clean.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(path)

# quick check
df.printSchema()
df.show(5, truncate=50)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+--------------------------------------------------+--------------+------+-------+----+----------+----+-----+--------------+-------------------------+----------------+------------------+------------+
|                                               App|      Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|                   Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------------------------------------+--------------+------+-----

In [77]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, to_date, initcap, lower, avg, round, when

# Spark session (reuse if already created)
spark = SparkSession.builder.appName("GPS").getOrCreate()

# Load the truly clean file you just made
path = "/Users/simranghai/Desktop/Google Play Store/googleplaystore_clean.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(path)

# ── 1. Robust numeric cleaning ──
df = (
    df
    # keep digits & dot for Price, cast to double
    .withColumn(
        "Price",
        regexp_replace(col("Price"), r"[^0-9.]", "").cast("double")
    )
    # keep digits only for Installs, cast to long
    .withColumn(
        "Installs",
        when(
            regexp_replace(col("Installs"), r"[^0-9]", "") == "",
            None
        ).otherwise(
            regexp_replace(col("Installs"), r"[^0-9]", "").cast("long")
        )
    )
)

# ── 2. Date & Category cleanup ──
df = (
    df
    .withColumn("Last Updated",
        to_date(col("Last Updated"), "MMMM d, yyyy")
    )
    .withColumn("Category",
        initcap(lower(regexp_replace(col("Category"), "_", " ")))
    )
)

# ── 3. Drop rows missing critical fields ──
df = df.filter(col("App").isNotNull() & col("Category").isNotNull())

# ── 4. Quick sanity check ──
df.printSchema()
df.show(5, truncate=50)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: long (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: date (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+--------------------------------------------------+--------------+------+-------+----+--------+----+-----+--------------+-------------------------+------------+------------------+------------+
|                                               App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|                   Genres|Last Updated|       Current Ver| Android Ver|
+--------------------------------------------------+--------------+------+-------+----+--------